In [1]:
import pandas as pd

# option to view all columns
pd.set_option('display.max_columns', None)

In [7]:
# test = pd.read_csv("data\processed\fourier_window_metrics_test.csv")
train = pd.read_csv("data/processed/fourier_window_metrics_train.csv")
test = pd.read_csv("data/processed/fourier_window_metrics_test.csv")

In [9]:
test = test.drop(columns=['clase'])

In [8]:
train

,nombre_archivo,clase,win_idx,t_start,t_end,duration,rms,var,zcr,crest_factor,peak_to_peak,skew_time,kurt_time,entropy_time,entropy_energy,f0_autocorr,harmonic_ratio,spec_centroid,spec_spread,spec_skew,spec_kurt,spec_rolloff85,spec_rolloff95,spec_flatness,spec_slope,spec_decrease,spec_crest,spec_entropy,f_peak,peak_prominence,band_0_250,band_250_500,band_500_1000,band_1000_2000,band_2000_4000,band_4000_8000,spec_flux,amp_p1,amp_p10,amp_p25,amp_p50,amp_p75,amp_p90,amp_p99
0,1.wav,1,0,0.00,0.500,0.500,0.027633,0.000764,0.221111,3.232535,0.175507,0.089720,-0.036967,1.968576e+00,3.162170,400.000000,0.551666,267.501636,435.221260,2.017326,6.095329,718.750000,1367.187500,0.128000,-0.000635,0.013052,326.536919,5.932677,29.296875,7.628709,0.733874,0.079229,0.094879,0.091913,1.053231e-04,0.0,NaN,-0.064548,-0.036209,-0.019012,-0.000031,0.018013,0.035873,0.067505
1,1.wav,1,1,0.25,0.750,0.500,0.026916,0.000724,0.221111,3.189361,0.162231,0.005357,-0.082236,1.933031e+00,3.259374,400.000000,0.571643,297.224631,422.685249,1.880251,5.921921,802.734375,1283.203125,0.132787,-0.000628,0.012143,243.345810,6.276421,29.296875,2.280243,0.653060,0.152724,0.111634,0.082580,1.622962e-06,0.0,0.555631,-0.063878,-0.035431,-0.018494,-0.000061,0.018410,0.034952,0.063322
2,1.wav,1,2,0.50,1.000,0.500,0.024307,0.000591,0.227114,3.142575,0.147430,-0.134318,-0.237424,1.796059e+00,3.285356,400.000000,0.486183,335.370900,411.339625,1.696729,5.615441,794.921875,1230.468750,0.174491,-0.000647,0.014224,166.857214,7.029711,29.296875,4.247886,0.561047,0.209589,0.146827,0.082501,3.574695e-05,0.0,0.507396,-0.058997,-0.032352,-0.016327,0.000763,0.017014,0.031821,0.052248
3,1.wav,1,3,0.75,1.250,0.500,0.049079,0.002408,0.312156,3.974546,0.367828,-0.020777,1.028778,2.675545e+00,2.655511,444.444444,0.723032,722.587847,353.596405,-0.789442,2.915499,945.312500,958.984375,0.083075,-0.000841,0.002472,51.461433,6.601750,29.296875,1.129199,0.159007,0.141890,0.666387,0.032704,1.181895e-05,0.0,0.973455,-0.125311,-0.059637,-0.026604,-0.000549,0.025436,0.058270,0.128571
4,1.wav,1,4,1.00,1.500,0.500,0.062430,0.003897,0.402701,3.124598,0.382690,0.024351,-0.396564,3.045189e+00,3.204536,500.000000,0.552083,782.667378,341.952590,-0.697266,3.798195,976.562500,1019.531250,0.076726,-0.000810,0.003824,56.903325,7.014019,29.296875,1.394615,0.114261,0.098410,0.672394,0.114896,3.933279e-05,0.0,0.585953,-0.131691,-0.082706,-0.045204,-0.001251,0.044098,0.083282,0.134588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16647,99.wav,4,82,20.50,21.000,0.500,0.660811,0.436385,0.560280,1.513293,1.999969,0.045996,-1.345988,5.836940e+00,3.251537,444.444444,0.357236,1146.509763,388.227528,0.352376,2.627032,1703.125000,1785.156250,0.217308,0.009047,-0.007033,70.046106,7.939235,876.953125,1.911584,0.013792,0.014443,0.456080,0.515405,2.793909e-04,0.0,0.570296,-0.998232,-0.933600,-0.625397,-0.045181,0.613022,0.926703,0.993870
16648,99.wav,4,83,20.75,21.250,0.500,0.619386,0.383544,0.582791,1.614501,1.999969,0.002268,-1.135565,5.672476e+00,3.117451,125.000000,0.250403,1305.220116,418.298244,-0.378231,2.261268,1740.234375,1833.984375,0.245478,0.012707,-0.002703,26.834620,8.239484,1634.765625,1.067659,0.012973,0.015863,0.265686,0.705334,1.439610e-04,0.0,0.551364,-0.998506,-0.921924,-0.539558,-0.023499,0.512924,0.898523,0.997358
16649,99.wav,4,84,21.00,21.500,0.500,0.364503,0.132518,0.271136,2.743460,1.999969,0.006959,1.890039,3.266429e+00,1.979594,333.333333,0.274413,1410.492603,384.369755,-0.916404,3.574764,1732.421875,1914.062500,0.213790,0.009752,-0.019921,33.122746,8.305801,1634.765625,1.029021,0.013544,0.007951,0.141554,0.836888,6.372942e-05,0.0,0.553597,-0.976411,-0.486774,-0.026253,-0.023560,-0.014915,0.438910,0.975111
16650,99.wav,4,85,21.25,21.750,0.500,0.022858,0.000009,0.000000,1.344440,0.021088,1.592223,5.109355,-1.442823e-12,3.303915,500.000000,0.815782,1.080741,22.395837,52

# Each row is a sample

In [10]:
# %pip install -U pandas numpy soundfile pyarrow
import os
from pathlib import Path
import numpy as np
import pandas as pd
import soundfile as sf  # fast header read
from typing import List

# --------------------
# CONFIG
# --------------------
CSV_PATH   = Path("results/window_metrics.csv")    # your table above
AUDIO_DIR  = Path("data/raw/Training_data")        # or Testing_data
OUT_DIR    = Path("results/per_sample")            # one parquet per audio
CLOSED     = "left"                                # interval closure: 'left' → [start, end)
FILL_MODE  = "drop"                                # 'drop' | 'pad_prev' | 'nearest_center'

META_COLS  = {"nombre_archivo","win_idx","t_start","t_end","duration"}  # keep others as features

OUT_DIR.mkdir(parents=True, exist_ok=True)

# --------------------
# HELPERS
# --------------------
def audio_info(path: Path):
    """Return (samplerate, frames). Avoid loads; uses header only."""
    info = sf.info(str(path))
    return info.samplerate, info.frames

def map_windows_to_samples(g: pd.DataFrame, sr: int, n_frames: int,
                           closed: str = "left", fill_mode: str = "drop") -> pd.DataFrame:
    """
    Vectorized interval assignment:
      - build IntervalIndex from [t_start, t_end)
      - get_indexer on all sample times t=i/sr
    fill_mode:
      - 'drop': drop samples falling outside any window
      - 'pad_prev': forward-fill using previous window index for uncovered samples
      - 'nearest_center': assign nearest window center (breaks strict interval semantics)
    """
    # Ensure sorted by t_start for stable indexing
    g = g.sort_values(["t_start","t_end"], kind="mergesort").reset_index(drop=True)

    # Sample-time grid
    t = np.arange(n_frames, dtype=np.float64) / float(sr)  # shape (N,)

    if fill_mode == "nearest_center":
        # Use window centers and asof-nearest
        centers = (g["t_start"].to_numpy() + g["t_end"].to_numpy()) * 0.5
        # pandas.merge_asof expects DataFrames
        df_left  = pd.DataFrame({"t": t})
        df_right = pd.DataFrame({"center": centers})
        df_right["win_idx_row"] = np.arange(len(g), dtype=np.int64)
        out = pd.merge_asof(df_left.sort_values("t"), 
                            df_right.sort_values("center"),
                            left_on="t", right_on="center", direction="nearest")
        row_idx = out["win_idx_row"].to_numpy()
        valid   = ~row_idx.isna()
        row_idx = row_idx.astype(np.int64, copy=False)
    else:
        # Interval membership
        iv = pd.IntervalIndex.from_arrays(g["t_start"].to_numpy(),
                                          g["t_end"].to_numpy(),
                                          closed=closed)
        row_idx = iv.get_indexer(t)  # -1 if no covering interval
        if fill_mode == "pad_prev":
            # replace -1 by previous valid assignment (forward-fill on the index array)
            # compute prefix max of (idx+1), then subtract 1
            # where first values are -1 → stay -1
            pos = row_idx + 1
            np.maximum.accumulate(pos, out=pos)
            pos[pos == 0] = 0  # remain 0 for leading -1s
            row_idx = pos - 1

        valid = row_idx >= 0

    # Filter to valid samples
    if not np.any(valid):
        return pd.DataFrame(columns=["nombre_archivo","sample_idx","t","sr","clase"])

    row_idx_valid = row_idx[valid]
    sample_idx    = np.flatnonzero(valid)
    t_valid       = t[valid]

    # Gather rows once; avoid per-sample joins
    g_take = g.iloc[row_idx_valid].reset_index(drop=True)

    # Columns
    keep_cols: List[str] = [c for c in g.columns]   # keep everything; or prune here
    out = g_take[keep_cols].copy()
    out.insert(0, "sample_idx", sample_idx.astype(np.int64, copy=False))
    out.insert(1, "t", t_valid)
    out.insert(2, "sr", int(sr))
    # nombre_archivo already present in g_take, ensure dtype stable
    return out

# --------------------
# MAIN
# --------------------
def windows_to_per_sample(csv_path: Path, audio_dir: Path, out_dir: Path,
                          closed: str = "left", fill_mode: str = "drop") -> None:
    df = pd.read_csv(csv_path)
    required = {"nombre_archivo","t_start","t_end"}
    missing  = required - set(df.columns)
    if missing:
        raise ValueError(f"CSV missing required columns: {sorted(missing)}")
    # normalize dtypes
    df["t_start"] = df["t_start"].astype(float)
    df["t_end"]   = df["t_end"].astype(float)

    for fname, g in df.groupby("nombre_archivo", sort=False):
        wav = (audio_dir / fname)
        if not wav.exists():
            print(f"[skip] audio not found: {wav}")
            continue
        sr, n_frames = audio_info(wav)
        out = map_windows_to_samples(g, sr, n_frames, closed=closed, fill_mode=fill_mode)
        if out.empty:
            print(f"[warn] no valid samples for {fname}")
            continue
        # Order columns: id → time → class → features
        first = ["nombre_archivo","sample_idx","t","sr"]
        others = [c for c in out.columns if c not in first]
        out = out[first + others]
        # Write one file per audio to keep RAM bounded
        out_path = out_dir / f"{Path(fname).stem}.parquet"
        out.to_parquet(out_path, index=False)
        print(f"[ok] {fname}: {len(out)} rows → {out_path}")

# Run
if __name__ == "__main__":
    windows_to_per_sample(CSV_PATH, AUDIO_DIR, OUT_DIR,
                          closed=CLOSED, fill_mode=FILL_MODE)


FileNotFoundError: [Errno 2] No such file or directory: 'results\\window_metrics.csv'

# Reduction of dimensionality with PCA

In [9]:
test

,nombre_archivo,win_idx,t_start,t_end,duration,rms,var,zcr,crest_factor,peak_to_peak,skew_time,kurt_time,entropy_time,entropy_energy,f0_autocorr,harmonic_ratio,spec_centroid,spec_spread,spec_skew,spec_kurt,spec_rolloff85,spec_rolloff95,spec_flatness,spec_slope,spec_decrease,spec_crest,spec_entropy,f_peak,peak_prominence,band_0_250,band_250_500,band_500_1000,band_1000_2000,band_2000_4000,band_4000_8000,spec_flux,amp_p1,amp_p10,amp_p25,amp_p50,amp_p75,amp_p90,amp_p99
0,1.wav,0,0.00,0.500,0.500,0.423467,0.178812,0.411206,2.359297,1.990814,0.006273,0.160461,5.431378,2.788730,363.636364,0.142217,1014.083956,307.431734,0.815783,5.452803,1199.218750,1777.343750,0.222943,0.002062,-0.021089,19.205854,8.408415,1017.578125,1.103684,0.010788,0.010238,0.448414,0.530271,2.897121e-04,0.0,NaN,-0.972054,-0.610358,-0.244019,-0.026123,0.185844,0.549069,0.960938
1,1.wav,1,0.25,0.750,0.500,0.397856,0.157745,0.518259,2.500358,1.986511,0.011637,0.149447,5.619568,3.030346,153.846154,0.134371,1039.206852,324.509156,0.626250,4.701847,1248.046875,1808.593750,0.220372,0.002735,-0.023553,18.704815,8.470851,1087.890625,1.284541,0.010942,0.009468,0.423502,0.555786,3.024054e-04,0.0,0.508992,-0.966437,-0.516873,-0.257950,-0.021271,0.202354,0.486121,0.931341
2,1.wav,2,0.50,1.000,0.500,0.260065,0.067054,0.514757,3.756238,1.894165,0.020097,1.253152,5.027592,3.131157,58.823529,0.116688,1103.936291,430.591127,0.338231,3.044213,1724.609375,1931.640625,0.278186,0.002819,-0.036195,17.033764,8.719487,0.000000,1.196258,0.023090,0.013149,0.410932,0.552163,6.651398e-04,0.0,0.638691,-0.722222,-0.330280,-0.166840,-0.021713,0.115799,0.283719,0.699496
3,1.wav,3,0.75,1.250,0.500,0.364762,0.132542,0.522261,2.717587,1.979187,0.005549,0.869586,5.409575,2.727826,125.000000,0.258194,1079.255642,375.168623,0.531409,3.298222,1423.828125,1847.656250,0.209922,0.003743,-0.025038,20.619610,8.391293,1115.234375,1.105850,0.010576,0.011416,0.391164,0.586701,1.422955e-04,0.0,0.657032,-0.959638,-0.466779,-0.189537,-0.022964,0.147591,0.430237,0.954034
4,1.wav,4,1.00,1.500,0.500,0.444518,0.197110,0.527264,2.231914,1.983398,0.003716,-0.293411,5.766640,3.086177,125.000000,0.254610,1058.159620,321.787236,0.572963,4.006885,1328.125000,1757.812500,0.203115,0.004022,-0.019795,16.140830,8.414697,1001.953125,1.060521,0.007271,0.007794,0.408366,0.576569,5.946691e-07,0.0,0.468370,-0.968478,-0.608786,-0.307411,-0.026169,0.253113,0.565094,0.963990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3632,9.wav,9,2.25,2.750,0.500,0.737819,0.543675,0.557279,1.323580,1.921875,0.048710,-1.847326,4.393690,3.321350,363.636364,0.243641,1088.540661,287.643786,-0.616994,5.600789,1341.796875,1490.234375,0.255057,0.003834,-0.011885,26.063617,8.249147,1107.421875,1.074922,0.017877,0.024426,0.151978,0.805679,4.034422e-05,0.0,0.542233,-0.906250,-0.828125,-0.796875,-0.121094,0.781250,0.812500,0.875078
3633,9.wav,10,2.50,3.000,0.500,0.746940,0.557887,0.530765,1.286499,1.906250,-0.002663,-1.876103,4.287557,3.320918,500.000000,0.407651,1047.234054,241.475956,-0.261694,7.164281,1214.843750,1419.921875,0.179970,0.002551,0.000896,44.041620,7.621848,1054.687500,1.542448,0.010333,0.017840,0.193118,0.778703,5.816747e-06,0.0,0.638502,-0.914062,-0.828125,-0.796875,-0.015625,0.781250,0.812500,0.890625
3634,9.wav,11,2.75,3.250,0.500,0.743183,0.552310,0.561281,1.293003,1.906250,-0.023225,-1.866715,4.399825,3.319109,333.333333,0.349485,1095.945484,287.609662,-0.177151,4.807556,1378.906250,1541.015625,0.188555,0.003154,0.002276,36.632981,7.863361,1054.687500,1.298957,0.011804,0.024253,0.256092,0.707831,2.027530e-05,0.0,0.620750,-0.906250,-0.828125,-0.789062,0.027344,0.781250,0.812500,0.882812
3635,9.wav,12,3.00,3.500,0.500,0.675093,0.455620,0.627314,1.423415,1.898438,0.023190,-1.683107,5.015812,3.245384,444.444444,0.422052,1200.540365,334.805691,-1.037257,4.916577,1394.531250,1714.843750,0.268452,0.006420,-0.002734

In [13]:
# if nombre_archivo is the same, then it is the same audio file, so we want to put the info sample row in the same rows together
train = train.sort_values(by=['nombre_archivo'])
train

,nombre_archivo,clase,win_idx,t_start,t_end,duration,rms,var,zcr,crest_factor,peak_to_peak,skew_time,kurt_time,entropy_time,entropy_energy,f0_autocorr,harmonic_ratio,spec_centroid,spec_spread,spec_skew,spec_kurt,spec_rolloff85,spec_rolloff95,spec_flatness,spec_slope,spec_decrease,spec_crest,spec_entropy,f_peak,peak_prominence,band_0_250,band_250_500,band_500_1000,band_1000_2000,band_2000_4000,band_4000_8000,spec_flux,amp_p1,amp_p10,amp_p25,amp_p50,amp_p75,amp_p90,amp_p99
0,1.wav,1,0,0.00,0.500,0.500,0.027633,0.000764,0.221111,3.232535,0.175507,0.089720,-0.036967,1.968576e+00,3.162170,400.000000,0.551666,267.501636,435.221260,2.017326,6.095329,718.750000,1367.187500,0.128000,-0.000635,0.013052,326.536919,5.932677,29.296875,7.628709,0.733874,0.079229,0.094879,0.091913,1.053231e-04,0.0,NaN,-0.064548,-0.036209,-0.019012,-0.000031,0.018013,0.035873,0.067505
594,1.wav,1,594,148.50,149.000,0.500,0.058067,0.003372,0.419210,3.406658,0.392975,-0.215741,0.059209,2.941736e+00,3.246142,333.333333,0.394960,794.460985,577.061859,0.353941,2.185416,1369.140625,1955.078125,0.145159,-0.000483,0.004458,141.123916,6.989603,29.296875,2.285491,0.223905,0.089266,0.362899,0.323813,1.171904e-04,0.0,0.513630,-0.144111,-0.075485,-0.037926,0.002609,0.041260,0.069687,0.125038
595,1.wav,1,595,148.75,149.250,0.500,0.062839,0.003949,0.407204,3.860387,0.451050,-0.132567,0.224490,3.047381e+00,3.183030,114.285714,0.348647,820.187802,537.596572,0.275785,2.320401,1386.718750,1951.171875,0.167042,-0.000529,0.004674,121.436833,7.438683,29.296875,2.556125,0.170601,0.139306,0.385983,0.304020,9.121234e-05,0.0,0.647906,-0.151150,-0.082169,-0.041649,0.001938,0.041321,0.079227,0.144353
596,1.wav,1,596,149.00,149.500,0.500,0.067408,0.004544,0.443222,3.808820,0.472992,-0.092791,0.203162,3.162039e+00,3.237081,114.285714,0.421741,863.753348,514.061191,0.228401,2.313420,1390.625000,1849.609375,0.177092,-0.000472,0.003579,66.052828,7.693247,29.296875,1.852916,0.119466,0.195614,0.358553,0.326271,9.642356e-05,0.0,0.520230,-0.163579,-0.086139,-0.042900,0.000381,0.043633,0.084818,0.157704
597,1.wav,1,597,149.25,149.750,0.500,0.055495,0.003080,0.426713,4.626402,0.472992,-0.045191,0.724682,2.886643e+00,3.155666,235.294118,0.429998,858.097754,521.874558,0.287670,2.408881,1380.859375,1953.125000,0.188405,-0.000461,0.004685,44.915446,7.572538,464.843750,1.106275,0.138150,0.173995,0.345314,0.342541,2.292334e-07,0.0,0.578050,-0.134561,-0.067084,-0.033936,-0.000168,0.035057,0.069397,0.136757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16588,99.wav,4,23,5.75,6.250,0.500,0.545533,0.297489,0.617809,1.833068,1.999969,0.056298,-0.820841,5.901975e+00,3.044900,400.000000,0.255965,1267.054723,365.239116,0.171174,3.860025,1763.671875,1962.890625,0.221781,0.010075,-0.005258,31.558666,8.123200,1226.562500,1.006835,0.013547,0.014201,0.103081,0.868804,3.659790e-04,0.0,0.579493,-0.990266,-0.793079,-0.400726,-0.015350,0.377556,0.799326,0.993047
16587,99.wav,4,22,5.50,6.000,0.500,0.649755,0.422073,0.625813,1.539041,1.999969,0.032863,-1.371200,5.896300e+00,3.293281,400.000000,0.156306,1259.814143,336.079567,-0.393716,4.384118,1638.671875,1830.078125,0.249362,0.013099,-0.003119,22.291194,8.525068,1269.531250,1.102477,0.015965,0.016719,0.087477,0.879787,5.332978e-05,0.0,0.592398,-0.996948,-0.891428,-0.623703,-0.026245,0.586754,0.891452,0.997471
16586,99.wav,4,21,5.25,5.750,0.500,0.616166,0.379540,0.632316,1.622939,1.999969,0.027822,-1.288620,5.960319e+00,3.279222,90.909091,0.133538,1276.787666,361.316371,-0.303238,3.527336,1689.453125,1876.953125,0.266969,0.013423,-0.003948,22.431388,8.596918,1076.171875,1.133509,0.015732,0.013109,0.131245,0.839914,1.930075e-07,0.0,0.599042,-0.993013,-0.855957,-0.577209,-0.015869,0.538620,0.850531,0.993487
16607,99.wav,4,42,10.50,11.000,0.500,0.614045,0.376892,0.512756,1.628545,1.999969,0.028598,-1.125582,5.837024e+00,3.201648,181.818182,0.2